In [91]:
import re
import time as t
import threading
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup

In [92]:
STRAVA_URL = 'https://www.strava.com'
password = r'1357WorkWork!'
p = '1357Worker!'
usernames = [
    ('spektork@post.bgu.ac.il'),
#     (r'cool_table@yahoo.com'),
#     (r'cool_table2@yahoo.com'),
#     (r'cool_table3@yahoo.com'),
#     (r'cool_table4@yahoo.com'),
#     (r'cool_table5@yahoo.com'),
]

username_backup = r'kev_backup@yahoo.com'
password = r'1357Backup!'

## Prepare chrome driver and ign in to Strava

In [93]:
def chrome_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('headless')

    # start chrome browser
    browser = webdriver.Chrome(options=options)

    browser.get(STRAVA_URL + '/login')
    email = browser.find_element_by_id("email")
    password = browser.find_element_by_id("password")

    email.send_keys('kev_backup@yahoo.com')
    password.send_keys('12345678')
    
    browser.find_element_by_id("login-button").click()
    return browser

In [102]:
semaphore = threading.Semaphore(len(usernames))
mutex = threading.Lock()

## Extract rider activities

In [103]:
class GetActivitiesThread(threading.Thread):        

    def __init__(self, links, username):
        self.activity_links = set()
        self.links = links
        self.username = username
        self.STRAVA_URL = 'https://www.strava.com'
        super(GetActivitiesThread, self).__init__()
        
    def open_driver(self):
        
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        
        self.browser = webdriver.Chrome(options=options)
        URL = self.STRAVA_URL + '/login'
        
        self.browser.get(URL)
        email = self.browser.find_element_by_id("email")
        password = self.browser.find_element_by_id("password")

        email.send_keys(self.username)
        password.send_keys('12345678')
        
        self.browser.find_element_by_id("login-button").click()
        t.sleep(1)
        print(self.browser.current_url)
       # print(f'thread: {self.name}, id: {browser.session_id}, logged in:  {browser.current_url}')
        
    def close_driver(self):
        self.browser.close()


    def run(self):
        
        semaphore.acquire()
        self.open_driver()

        prev = set()
        curr = set()
        for link in self.links:
            
            self.browser.get(link)
            t.sleep(5)
            if self.browser.current_url == 'https://www.strava.com/login': # logged out ...
                print(f'thread: {self.name}, logged out...')
                self.close_driver()
                semaphore.release()
                return
            timeout = t.time() + 15
            new_curr = set()
            
            while curr == prev:
                
               # print(f'thread: {self.name}, {self.browser.current_url}')   
                t.sleep(1)
                graph_soup = BeautifulSoup(self.browser.page_source, 'html.parser')
                group_feed_activities = graph_soup.find_all('div', {'class': 'feed-entry group-activity'})
                normal_activities = graph_soup.find_all('div', {'class': 'activity feed-entry entity-details'})

                for group_feed_activity in group_feed_activities:
                    # loop through all group activities
                    group_feed_entry_body = group_feed_activity.find('div', {'class': 'entry-body'})
                    group_feed_link = group_feed_entry_body.find('a')
                    new_curr.add(self.STRAVA_URL + group_feed_link.get('href'))
                
                for normal_activity in normal_activities:
                    # loop through all normal activities
                    normal_activity_entry_body = normal_activity.find('div', {'class': 'entry-body'})
                    normal_activity_link = normal_activity_entry_body.find('a')
                    new_curr.add(self.STRAVA_URL + normal_activity_link.get('href'))
                curr = new_curr
                
                if t.time() > timeout:
                    break
                
            prev = curr
            
            self.activity_links = self.activity_links.union(curr)
            
        self.close_driver()
        semaphore.release()


In [96]:
def extract_activities(url, years, months):
    activity_links = set()
    actvity_links_list = []
    STRAVA_URL = 'https://www.strava.com'
    browser.get(url)
    
    # Click on 'monthly' button so that the date inteverals would be smaller
    monthly_xpath = '//*[@id="interval-graph-controls"]/ul/li[2]/ul/li[2]/a'
    interval_xpath = '//*[@id="interval-graph-columns"]/div/ul[2]'

    soup = BeautifulSoup(browser.page_source, 'html.parser')
    date_intervals = soup.find('div', {'class': 'drop-down-menu drop-down-sm enabled'})
    date_links = date_intervals.find_all('a')
    
    threads_links = []
    interval_soups = []
    i = 0
    prev_url = browser.current_url
    for date_link in date_links:
        
        link = STRAVA_URL + date_link['href']
        link = link.replace('week', 'month')
        interval1 = re.search("interval=.*&",link).group(0)
        year = int(interval1[-7:-3])
        
        if year not in years:
            continue
        threads_links.append([])
        for month in range(7,13):
            
            if month not in months:
                continue
                
            month_string = str(month) if month >= 10 else f'0{month}'
            interval2 = interval1.replace(str(year)+interval1[-3:-1], str(year-1)+month_string)
            threads_links[i].append(link.replace(interval1, interval2))
            
        for month in range(1,8):
            
            if month not in months:
                continue
                
            interval2 = interval1.replace(str(year)+interval1[-3:-1], str(year)+f'0{month}')
            threads_links[i].append(link.replace(interval1, interval2))

       
        i += 1
        
    return threads_links

In [97]:
browser = chrome_driver()
browser.current_url

'https://www.strava.com/onboarding'

In [104]:
def extract_athelete_activities(url, years=[2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014], months=[1,2,3,4,5,6,7,8,9,10,11,12]):
    threads_links = extract_activities(url, years, months)
    threads = []
    activity_links = set()
    
    i = 0
    for links in threads_links:
        thread = GetActivitiesThread(links, usernames[i % len(usernames)])
        thread.name = str(i)
        threads.append(thread)
        thread.start()
        i += 1

    for thread in threads:
        thread.join()
        print(f'thread: {thread.name}, links: {len(thread.activity_links)}')
        activity_links = activity_links.union(thread.activity_links)
    return activity_links

In [105]:
se = extract_athelete_activities('https://www.strava.com/athletes/9207204', years=[2020],months=[1,2,3])


https://www.strava.com/onboarding
thread: 0, links: 0


In [106]:
se

set()